In [1]:
# Caso ainda não tenha instalado,
# !pip install tqdm
# !pip install tqdm_joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

# Importando as bibliotecas para paralelização e progress bar
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

c:\Users\Ruy\anaconda3\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Carregar os dados tratados
df = pd.read_csv("../Data/Stroke.csv")

In [3]:
# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

In [4]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
# Configurando o classificador KNN
knn_model = KNeighborsClassifier()

# Definindo o grid de hiperparâmetros para o KNN
param_grid = {
    'n_neighbors': [ 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1 para Manhattan, 2 para Euclidiana
}

# Configurando o GridSearchCV com validação cruzada de 5 folds e paralelização (n_jobs=-1)
grid_search = GridSearchCV(
    estimator=knn_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0  # O tqdm exibirá o progresso, então podemos deixar verbose=0
)

In [6]:
# Calculando o total de iterações: número de combinações * número de folds
total_iter = grid_search.cv * np.prod([len(v) for v in param_grid.values()])
print(f"Total de iterações estimadas: {total_iter}")

# Realizando o Grid Search com tqdm_joblib para monitorar o progresso
with tqdm_joblib(tqdm(desc="GridSearchCV", total=total_iter)) as progress_bar:
    grid_search.fit(X_train, y_train)

Total de iterações estimadas: 320


GridSearchCV:   0%|          | 0/320 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

In [7]:
print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

Melhores hiperparâmetros: {'algorithm': 'kd_tree', 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
Melhor acurácia: 0.920099489473386


In [8]:
# Avaliação do modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred))


Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      0.87      0.93       951
           1       0.88      1.00      0.94       943

    accuracy                           0.93      1894
   macro avg       0.94      0.93      0.93      1894
weighted avg       0.94      0.93      0.93      1894


Matriz de Confusão:
 [[824 127]
 [  0 943]]
